SimpleITK based Segmentation

In [1]:
# Importing SimpleITK library for medical image processing
import SimpleITK as sitk
# Importing numpy library for array operations
import numpy as np  
# Importing helper functions (assuming they are defined in helpers.py)
from helpers import *  
# Path to the sample MRI data file
sample_mri_path = 'MRNet-v1.0/train/coronal/0002.npy' 
# Loading the MRI data file into a numpy array 
mri_array = np.load(sample_mri_path) 


In [2]:
 # Convert the numpy array `mri_array` to a SimpleITK image object `mri_image`
mri_image = sitk.GetImageFromArray(mri_array) 


In [3]:
# Display a 3D visualization of the MRI array using a helper function
explore_3D_array(mri_array)


(32, 256, 256)

In [4]:
# Display a 3D visualization of the MRI array using a helper function
explore_3D_array(mri_array)explore_3D_array(mri_array)

interactive(children=(IntSlider(value=15, description='SLICE', max=31), Output()), _dom_classes=('widget-inter…

In [5]:
# Apply a segmentation threshold to the MRI image
seg = mri_image > 200

# Overlay the thresholded segmentation on the MRI image
thresh_img = sitk.LabelOverlay(mri_image, seg)

# Convert the thresholded image back to an array for further processing
thresh_array = sitk.GetArrayFromImage(thresh_img)


In [6]:
# Visualize the 3D array using a custom function
explore_3D_array(thresh_array)


interactive(children=(IntSlider(value=15, description='SLICE', max=31), Output()), _dom_classes=('widget-inter…

In [7]:
# Create a binary threshold segmentation of the MRI image
seg = sitk.BinaryThreshold(mri_image, lowerThreshold=100, upperThreshold=400, insideValue=1, outsideValue=0)

# Overlay the original MRI image with the segmentation to create a labeled overlay image
thresh_img = sitk.LabelOverlay(mri_image, seg)

# Convert the labeled overlay image to a numpy array for visualization
thresh_array = sitk.GetArrayFromImage(thresh_img)


In [8]:
# Visualize the 3D array (thresh_array) using a custom function
explore_3D_array(thresh_array)


interactive(children=(IntSlider(value=15, description='SLICE', max=31), Output()), _dom_classes=('widget-inter…

In [9]:
# Apply Otsu thresholding to the MRI image
otsu_filter = sitk.OtsuThresholdImageFilter()
otsu_filter.SetInsideValue(0)
otsu_filter.SetOutsideValue(1)
seg = otsu_filter.Execute(mri_image)

# Overlay the segmented image with the original MRI image
thresh_img = sitk.LabelOverlay(mri_image, seg)

# Convert the labeled overlay image to a numpy array
thresh_array = sitk.GetArrayFromImage(thresh_img)

# Visualize the segmented 3D array using a custom function
explore_3D_array(thresh_array)

# Print the computed threshold value from the Otsu filter
print(otsu_filter.GetThreshold())


interactive(children=(IntSlider(value=15, description='SLICE', max=31), Output()), _dom_classes=('widget-inter…

85.0


In [10]:
print(mri_array.shape)

(32, 256, 256)


In [11]:
# Define a seed point for region growing
seed = (128, 128, 18)

# Create a binary image for segmentation with the same size and metadata as the MRI image
seg = sitk.Image(mri_image.GetSize(), sitk.sitkUInt8)
seg.CopyInformation(mri_image)

# Set the seed point to value 1 in the binary image
seg[seed] = 1

# Perform binary dilation on the segmented image using a (9,9,9) kernel
seg = sitk.BinaryDilate(seg, (9, 9, 9))

# Overlay the segmented image with the original MRI image
thresh_img = sitk.LabelOverlay(mri_image, seg)

# Convert the labeled overlay image to a numpy array
thresh_array = sitk.GetArrayFromImage(thresh_img)

# Visualize the segmented 3D array using a custom function
explore_3D_array(thresh_array)


interactive(children=(IntSlider(value=15, description='SLICE', max=31), Output()), _dom_classes=('widget-inter…

In [12]:
# Define a seed point for the confidence connected segmentation
seed = (128, 150, 18)

# Perform confidence connected segmentation starting from the seed point
seg = sitk.ConfidenceConnected(mri_image,
                               seedList=[seed],
                               numberOfIterations=1,
                               multiplier=2.5,
                               initialNeighborhoodRadius=1,
                               replaceValue=1)

# Overlay the segmented image with the original MRI image
thresh_img = sitk.LabelOverlay(mri_image, seg)

# Convert the labeled overlay image to a numpy array
thresh_array = sitk.GetArrayFromImage(thresh_img)

# Visualize the segmented 3D array using a custom function
explore_3D_array(thresh_array)


interactive(children=(IntSlider(value=15, description='SLICE', max=31), Output()), _dom_classes=('widget-inter…

In [13]:
# Compute the gradient magnitude of the MRI image using a recursive Gaussian filter
feature_img = sitk.GradientMagnitudeRecursiveGaussian(mri_image, sigma=0.5)

# Compute the bounded reciprocal of the gradient magnitude image (speed image)
speed_img = sitk.BoundedReciprocal(feature_img)

# Convert the speed image to a numpy array for visualization
thresh_array = sitk.GetArrayFromImage(speed_img)

# Visualize the 3D array using a custom function
explore_3D_array(thresh_array)


interactive(children=(IntSlider(value=15, description='SLICE', max=31), Output()), _dom_classes=('widget-inter…

In [14]:
# Initialize the Fast Marching filter and set trial points and stopping value
fm_filter = sitk.FastMarchingBaseImageFilter()
fm_filter.SetTrialPoints([seed])
fm_filter.SetStoppingValue(1000)

# Execute the Fast Marching filter on the speed image
fm_img = fm_filter.Execute(speed_img)

# Threshold the Fast Marching image to create a segmentation
thresh_img = sitk.Threshold(fm_img,
                            lower=0.0,
                            upper=fm_filter.GetStoppingValue(),
                            outsideValue=fm_filter.GetStoppingValue() + 1)

# Convert the thresholded image to a numpy array for visualization
thresh_array = sitk.GetArrayFromImage(thresh_img)

# Visualize the 3D array using a custom function
explore_3D_array(thresh_array)


interactive(children=(IntSlider(value=15, description='SLICE', max=31), Output()), _dom_classes=('widget-inter…

In [15]:
# Define the seed point and initialize the segmentation image
seed = (128, 128, 18)
seg = sitk.Image(mri_image.GetSize(), sitk.sitkUInt8)
seg.CopyInformation(mri_image)
seg[seed] = 1

# Apply binary dilation to the segmentation image
seg = sitk.BinaryDilate(seg, (3, 3, 3))

# Compute statistics within the segmented region using LabelStatisticsImageFilter
stats = sitk.LabelStatisticsImageFilter()
stats.Execute(mri_image, seg)

# Calculate lower and upper thresholds based on statistics
factor = 3.5
lower_threshold = stats.GetMean(1) - factor * stats.GetSigma(1)
upper_threshold = stats.GetMean(1) + factor * stats.GetSigma(1)

# Print the computed thresholds
print(lower_threshold, upper_threshold)


78.5598429607918 225.5295425140685


In [16]:
# Initialize the Signed Maurer Distance Map using the segmented image
init_ls = sitk.SignedMaurerDistanceMap(seg, insideIsPositive=True, useImageSpacing=True)



In [17]:
# Configure the Level Set segmentation filter
lsFilter = sitk.ThresholdSegmentationLevelSetImageFilter()
lsFilter.SetLowerThreshold(lower_threshold)
lsFilter.SetUpperThreshold(upper_threshold)
lsFilter.SetMaximumRMSError(0.02)
lsFilter.SetNumberOfIterations(1000)
lsFilter.SetCurvatureScaling(0.5)
lsFilter.SetPropagationScaling(1)
lsFilter.ReverseExpansionDirectionOn()

# Execute the Level Set segmentation with the initialized distance map and MRI image
ls = lsFilter.Execute(init_ls, sitk.Cast(mri_image, sitk.sitkFloat32))

# Print the configured filter for reference
print(lsFilter)


In [ ]:
# Generate a label overlay image based on the Level Set segmentation result
thresh_img = sitk.LabelOverlay(mri_image, ls > 0)

# Convert the label overlay image to a NumPy array for visualization
thresh_array = sitk.GetArrayFromImage(thresh_img)

# Explore the 3D array to visualize the segmented regions
explore_3D_array(thresh_array)

# Comment: Visualize the MRI image with the segmented regions highlighted based on the Level Set result.


interactive(children=(IntSlider(value=15, description='SLICE', max=31), Output()), _dom_classes=('widget-inter…